# Work sheet

**Lương** - lấy link track và playlist, user\
**Long & Khuê** - lấy thông tin của tracks và user, playlist  &  thiết kế cấu trúc lưu file csv.

# Import Necessary Library

## Import `selenium` package

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re
print("Import Complete!!!")

Import Complete!!!


## ***Project pipeline***

**Crawling order:**\
<font color='green'>track -> playlist -> user -> track </font>

- Crawling all track's links that presence in [soundcloud](https://soundcloud.com/) page.
- Crawling all playlist that include tracks we have crawled.
- Crawling all tracks that included in all playlist we have crawled.
- Crawling all users of each track.

## Initialize a webdriver
Use cell below to initialize a `webdriver`.

<font color='green'> **We need to:**</font>
- Find a button **Accept all cookies** and excecute a `click()` operation.
- If **Accept all cookies** button isn't presented we pass to crawling phase.

In [6]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://soundcloud.com/')
driver.maximize_window()

try:
    accept_cookies = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
    )
    accept_cookies.click()
except:
    pass

time.sleep(3)

````python
accept_cookies = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
)
````
<font color='blue'>We use this block of code to wait till **a button specify by `ID` show up** with a specify time, otherwise we skip **that button**.</font>

## Extract Link Phase
- Extract link of <font color='green'>**Tracks**
- Extract link of <font color='green'>**Playlists**
- Extract link of <font color='green'>**Users**

### Get some playlist's links

***write function to scroll page down***

In [7]:
def scroll_page_n(d, n: int):
    '''scroll page down n times'''
    # get current height of the page
    last_height = d.execute_script("return document.body.scrollHeight")
    
    if n <= 0:
        while True:
            # scroll page to end
            ele = d.find_element_by_tag_name('body')
            ele.send_keys(Keys.END)
            
            # wait for loading page
            time.sleep(2.5)
            
            # get current page's height
            new_height = d.execute_script('return document.body.scrollHeight')
            
            # check if we reach end of page
            if new_height == last_height:
                break
            last_height = new_height
    else:
        for i in range(n):
            # scroll page to end
            ele = d.find_element_by_tag_name('body')
            ele.send_keys(Keys.END)
            
            # wait for loading page
            time.sleep(1)

`execute_script("return document.body.scrollHeight")` is used to execute a `java script` code.

### Get get link of all playlists that present in [soundcloud](https://soundcloud.com/)

In [4]:
origin_tracks = [i.get_attribute('href') for i in driver.find_elements_by_xpath('//li/div/div/a')]
play_list_links = []

for i in origin_tracks:
    # check the if we have crawled 1000 records
    if len(play_list_links) > 1000:
        break
    
    # request a track's page
    driver.get(i)
        
    # wait for loading page (2 seconds)
    time.sleep(2)
    
    # find the "Not Right Now" button and execute click() operation otherwise pass.
    try:
        not_now = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[4]/div/div/div/div[2]/button[1]'))
        )
        not_now.click()
        time.sleep(1)
    except:
        pass
    
    # find the "In Playlist" button and execute click() operation in case have banner.
    try:
        in_playlist = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[3]/div[2]/div[1]/article[5]/a/h3/span[2]'))
        )
        in_playlist.click()
    except:
        pass
    
    # find the "In Playlist" button and execute click() operation in case not have banner
    # otherwise remove track from list.
    try:
        in_playlist = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[3]/div[2]/div[1]/article[4]/a/h3/span[2]'))
        )
        in_playlist.click()
    except:
        origin_tracks.remove(i)
        continue
    
    # scroll page down for loading all playlists.
    scroll_page_n(driver, 350)
    time.sleep(1.5)
    # find all playlist links.
    for i in driver.find_elements_by_xpath('//div/div/div[2]/div/div/ul/li/div/div/div[1]/a'):
        if i not in play_list_links:
            play_list_links.append(i.get_attribute('href'))
            if len(play_list_links) > 1000:
                break

In [5]:
playlist_track = dict()
for i in play_list_links:
    playlist_track[i] = []

- *all links we have extracted.*

In [6]:
play_list_links

['https://soundcloud.com/m-t-ongg/sets/uhm',
 'https://soundcloud.com/duy-linh-570362939/sets/chill',
 'https://soundcloud.com/duy-linh-570362939/sets/lofi',
 'https://soundcloud.com/ozeekxng/sets/prod-by-ozee-kxng',
 'https://soundcloud.com/user-360204099/sets/winno',
 'https://soundcloud.com/mai-nguy-n-880905326/sets/soul',
 'https://soundcloud.com/van-khanh-le-440559858/sets/van-khah',
 'https://soundcloud.com/nguy-n-anh-t-750293006/sets/nh-c',
 'https://soundcloud.com/hashou-shimazaki/sets/qwerty',
 'https://soundcloud.com/tu-n-v-239764509/sets/new-private-playlist',
 'https://soundcloud.com/truong-huu-nguyen-122242657/sets/chill-pills',
 'https://soundcloud.com/minh-chau-275968352/sets/love',
 'https://soundcloud.com/m-nh-l-954425611/sets/chill',
 'https://soundcloud.com/l-trung-ngh-a-303059230/sets/maaddie',
 'https://soundcloud.com/hien-ta-168463711/sets/st2efp3dxpyi',
 'https://soundcloud.com/minh-t-ph-m-501472597/sets/youngboi-si-tinh-vibe',
 'https://soundcloud.com/vi-t-l-m-9

- *number of playlists*

In [7]:
len(play_list_links)

1001

### Get link of all tracks of each playlist

In [14]:
track_links = []

for i in play_list_links:
    # get page of each playlist
    driver.get(i)
            
    # wait to load page
    time.sleep(1)
    
    # get link of all tracks from ith playlist
    tracks = driver.find_elements_by_xpath('//div/div[3]/div[1]/div/div[2]/div[2]/div/div[3]/div/ul/li/div/div[3]/a[2]')
    
    track_buffer = []
    for i in tracks:
        link = i.get_attribute('href')
        if link not in track_links:
            track_buffer.append(link)
    
    # just append the first 3 links in playlist
    track_links.extend(track_buffer[:3])
    track_buffer.clear()

- *all links we have extracted*

In [15]:
track_links

['https://soundcloud.com/khoidarealest/officialmp3-benh-cua-anh?in=m-t-ongg/sets/uhm',
 'https://soundcloud.com/jessejostarkmusic/mystery?in=m-t-ongg/sets/uhm',
 'https://soundcloud.com/lostlowna/feeling-warm-in-the-rain?in=m-t-ongg/sets/uhm',
 'https://soundcloud.com/thu-th-o-v-ng/co-em-doi-bong-vui-chillies-official-video?in=m-t-ongg/sets/uhm',
 'https://soundcloud.com/zarasteal/love-song-lana-del-rey?in=m-t-ongg/sets/uhm',
 'https://soundcloud.com/3hahustlin/chieu-tim-damd?in=duy-linh-570362939/sets/chill',
 'https://soundcloud.com/nanangx/tinh-dang-nhu-ly-ca-phe-ngo-va-nan?in=duy-linh-570362939/sets/chill',
 'https://soundcloud.com/arkarecords/anh-khong-the-gi-dau-anh-lam-aktgd-phuc-du?in=duy-linh-570362939/sets/chill',
 'https://soundcloud.com/double-blue/killic-x-dblue-chi-con-ta-va-ta-giua-troi-prod-by-plantplantworld-official-audio?in=duy-linh-570362939/sets/chill',
 'https://soundcloud.com/nguyen-pham-phuong-anh-919045202/wean-ai-biet?in=duy-linh-570362939/sets/chill',
 'https

- *number of tracks*

In [16]:
len(track_links)

4563

### Get link of user of each track

In [ ]:
user_links = []

for i in track_links:
    # Get page
    driver.get(i)
    try:
        user = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[2]/div[1]/div/div[2]/div/h2/a'))
        )
        link = user.get_attribute('href')
    except:
        pass
    
    if link not in user_links:
        user_links.append(link)
    
    if len(user_links) > 1000:
        break

- *all links we have extracted*

In [18]:
user_links

['https://soundcloud.com/khoidarealest',
 'https://soundcloud.com/jessejostarkmusic',
 'https://soundcloud.com/lostlowna',
 'https://soundcloud.com/thu-th-o-v-ng',
 'https://soundcloud.com/zarasteal',
 'https://soundcloud.com/3hahustlin',
 'https://soundcloud.com/nanangx',
 'https://soundcloud.com/arkarecords',
 'https://soundcloud.com/double-blue',
 'https://soundcloud.com/nguyen-pham-phuong-anh-919045202',
 'https://soundcloud.com/hoanglongnger',
 'https://soundcloud.com/lopepham',
 'https://soundcloud.com/fujishen',
 'https://soundcloud.com/ericthe17dopevcl',
 'https://soundcloud.com/winnodamidside',
 'https://soundcloud.com/lomaxindacut',
 'https://soundcloud.com/tripboidlg',
 'https://soundcloud.com/tecawiz',
 'https://soundcloud.com/chxxx88',
 'https://soundcloud.com/dewie_sayhiii',
 'https://soundcloud.com/youngbo5onfire',
 'https://soundcloud.com/da-willistic',
 'https://soundcloud.com/yourtrvsh999',
 'https://soundcloud.com/litgirldarkstar',
 'https://soundcloud.com/martingarr

- *number of users*

In [19]:
len(user_links)

1294

## Extract information phase
<font color='green'>**Using lists of link that we have extracted**<font>

### Get Information of each Track

<font color='green'>***Initialize dict to store information***

In [10]:
tracks = dict()
for i in 'name', 'date_post', 'plays', 'likes', 'reposts', 'genre':
    tracks[i] = []

<font color='green'>***Extract information Function***

In [11]:
def extract_track(link):
    """
    Extract information about a track from existing link.
    :param link: specify a link to request
    """ 
    # request a page
    driver.get(link)
    # wait for loading page
    time.sleep(2)
    
    # get number plays
    try:
        plays = driver.find_element_by_xpath('//ul/li[1]/span/span[1]').text.replace(' plays', '')
    except:
        plays = 'None'
    
    # date post
    try:
        date_post = driver.find_element_by_xpath('//div[3]/div/time').get_attribute('title').replace('Posted on ', '')
    except:
        date_post = 'None'
        
    # number likes
    try:
        likes = driver.find_element_by_xpath('//div/ul/li[2]/a/span[2]').text
    except:
        likes = 'None'
    
    # number of reposts
    try:
        reposts = driver.find_element_by_xpath('//div[2]/ul/li[3]/a/span[2]').text
    except:
        reposts = 'None'
    
    # name
    name = driver.find_element_by_xpath('//div/div/h1/span').text
    
    # get genre if it have otherwise return none
    genre = 'None'
    try:
        genre = driver.find_element_by_xpath('//div[2]/div[3]/a/span').text
    except:
        genre = 'None'
    
    return [name, date_post, plays, likes, reposts, genre]

In [12]:
for link in track_links:
    buffer = extract_track(link)
    for i, j in zip(tracks.keys(), range(len(buffer))):
        tracks[i].append(buffer[j])

In [13]:
track_csv = pd.DataFrame(tracks)

In [14]:
track_csv.to_csv('Crawl_data/track.csv', na_rep = 'None')

In [15]:
track_csv.head(5)

,name,date_post,plays,likes,reposts,genre
0,Bệnh của anh - Khói x Two,24 December 2015,"4,611,021",37.1K,"3,228",vietdreamerz
1,Mystery,2 November 2018,"58,847",662,50,Alternative Rock
2,feeling warm in the rain,3 June 2019,"19,276",331,25,Chillhop
3,Co Em Doi Bong Vui Chillies Official Video,7 February 2020,"420,368","2,812",95,Pop
4,Love Song- Lana Del Rey,8 September 2019,"33,867",797,10,Indie


### Meaning of each Column
- **name**: name of track.
- **date_post**: date when the track is posted.
- **plays**: number of plays.
- **likes**: number of likes.
- **reposts**: number of reposts.
- **genre**: genre of track.

### Get Information of each Playlist

<font color='green'>***Initialize dict to store information***

In [50]:
playlists = dict()
for i in 'name', 'date_post', 'tracks', 'likes', 'reposts', 'user', 'link':
    playlists[i] = []

<font color='green'>***Extract information Function***

In [53]:
def extract_playlist(link):
    like = ''
    repost = ''
    
    
    # request a page
    driver.get(link)
    
    # wait for loading page
    time.sleep(2)
    
    # get number tracks
    scroll_page_n(driver, -1)
    temp = driver.find_elements_by_xpath('//*[@id="content"]/div/div[3]/div[1]/div/div[2]/div[2]/div/div[3]/div/ul/li/div/div[2]/span')
    temp2 = len(temp)
    tracks = str(temp2)
    
    #date post
    date_post = driver.find_element_by_xpath('//div[3]/div/time').get_attribute('title').replace('Posted on ', '')
    
    # find likes,reposts
    buffer = None
    try:
        buffer = driver.find_elements_by_xpath('//*[@id="content"]/div/div[3]/div[1]/div/div[1]/div/div/div/ul/li/a/span')
    except:
        pass
    #number likes,reposts
    if(len(buffer) == 4):
        if(buffer[0].text == 'View all likes'):
            like = buffer[1].text
        elif(buffer[0].text == 'View all reposts'):
            repost = buffer[1].text
        if(buffer[2].text == 'View all likes'):
            like = buffer[3].text
        elif(buffer[2].text == 'View all reposts'):
            repost = buffer[3].text
    elif(len(buffer) == 2):
        if(buffer[0].text == 'View all likes'):
            like = buffer[1].text
            repost = 'None'
        elif(buffer[0].text == 'View all reposts'):
            like = 'None'
            repost = buffer[1].text        
    else:
        like = 'None'
        repost = 'None'
    
    # name
    name = driver.find_element_by_xpath('//div/div/h1/span').text
    
    #user:
    user = driver.find_element_by_xpath('//div/div/div[2]/div[1]/h3/a/span').text
    
    return [name, date_post, tracks, like, repost, user, link]

In [54]:
for link in play_list_links:
    buffer = extract_playlist(link)
    for i, j in zip(playlists.keys(), range(len(buffer))):
        playlists[i].append(buffer[j])

In [56]:
playlist_csv = pd.DataFrame(playlists)

In [58]:
playlist_csv.to_csv('Crawl_data/playlist.csv')

In [60]:
playlist_csv.head(5)

,name,date_post,tracks,likes,reposts,user,link
0,uhm,26 October 2020,117,2,None,jimbo,https://soundcloud.com/m-t-ongg/sets/uhm
1,Chill,16 August 2020,189,7,1,Duy Linh,https://soundcloud.com/duy-linh-570362939/sets...
2,Lofi,6 March 2021,80,3,1,Duy Linh,https://soundcloud.com/duy-linh-570362939/sets...
3,Produced by Ozee Kxng,21 February 2021,19,5,1,@ozeekxng,https://soundcloud.com/ozeekxng/sets/prod-by-o...
4,winno,13 May 2021,5,None,None,User 360204099,https://soundcloud.com/user-360204099/sets/winno


### Meaning of each Column
- **name**: name of playlist.
- **date_post**: date when the playlist is posted.
- **tracks**: number of tracks included in playlis.
- **likes**: number of likes.
- **reposts**: number of reposts.
- **user**: the playlist's owner.
- **link**: link of the user.

### Get Information of each User

<font color='green'>***Initialize dict to store information***

In [10]:
users = dict()
for i in 'username', 'realname', 'place', 'subscription', 'followers', 'following', 'tracks', 'likes', 'comments':
    users[i] = []

<font color='green'>***Extract information Function***

In [14]:
def extract_user(link):
    driver.get(link)
    time.sleep(2)
    
    username = ''
    realname = ''
    place = ''
    subscription = ''
    followers = ''
    following = ''
    tracks = ''
    likes = 'None'
    comments = 'None'
    buffer = None
    buffer = driver.find_elements_by_class_name("profileHeaderInfo__userName ")
    if len(buffer) != 0:
        username = buffer[0].text
        if 'Verified' in username:
            username = re.sub('Verified', '', username)
    else:
        username = 'None'
    buffer = None
    buffer = driver.find_elements_by_class_name("profileHeaderInfo__additional ")
    if len(buffer) == 2:
        realname = 'None'
        place = buffer[0].text
        subscription = buffer[1].text
        if subscription == '':
            subscription = 'Basic'
    elif len(buffer) == 3:
        realname = buffer[0].text
        place = buffer[1].text
        subscription = buffer[2].text
        if subscription == '':
            subscription = 'Basic'
    elif len(buffer) == 1:
        realname = 'None'
        place = 'None'
        subscription = buffer[0].text
        if subscription == '':
            subscription = 'Basic'
    buffer = None
    buffer = driver.find_elements_by_class_name("infoStats__value ")
    for i in range(0,len(buffer)):
        buffer[i] = buffer[i].text
        if buffer[i] =='':
            buffer[i] = 'None'
    followers = buffer[0]
    following = buffer[1]
    tracks = buffer[2]

    buffer = None
    buffer = driver.find_elements_by_xpath('//a/h3/span[2]')
    for i in buffer:
        if 'Fans also like' in i.text:
            continue
        
        if 'likes' in i.text:
            likes = i.text.replace(' likes', '')
        elif 'like' in i.text:
            likes = i.text.replace(' like', '')

        if 'comments' in i.text:
            comments = i.text.replace(' comments', '')
        elif 'comment' in i.text:
            comments = i.text.replace(' comment', '')
                
    return [username, realname, place, subscription, followers, following, tracks, likes, comments]

In [34]:
for link in user_links:
    temp = extract_user(link)
    for i, j in zip(users.keys(), range(len(temp))):
        users[i].append(temp[j])

In [35]:
user_csv = pd.DataFrame(users)

In [36]:
user_csv.to_csv('Crawl_data/user.csv')

In [38]:
user_csv.head(5)

,username,realname,place,subscription,followers,following,tracks,likes,comments
0,khói,None,dalat,Pro Unlimited,145K,6,81,61,152
1,Jesse Jo Stark,Jesse Jo Stark,Los Angeles,Basic,"1,755",0,19,None,1
2,lowna,None,Viet Nam,Basic,656,4,13,234,70
3,maybaygiay,May Bay Giay,"Ha Noi, Viet Nam",Basic,70,34,37,286,16
4,zarasteal,None,zara,Basic,35,6,17,46,26


### Meaning of each Column
- **username**: user name of user.
- **realname**: real name of user.
- **place**: location of user.
- **subscription**: type of user.
- **followers**: number of followers.
- **following**: number of following.
- **tracks**: number of tracks.
- **likes**: number of likes.
- **comments**: number of commnets.

# <font color='red'>***The End***